# Capstone project - Applied Data Science

## Week 3 - Assignment Part 1

Welcome to Capstone Project for Applied Data Science course on coursera powered by IBM.

This notebook is for the Assignment in Week 3 Part 1

The goal of the assignment is to read the Totonto city postal codes from Wikipedia and create a dataframe ready for clustering assignment in Part 2 and Part 3


In [39]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

Prepare Wikipedia URL to read postal codes for Totonto city  
Use Beautiful Soup to read the webpage

In [40]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Read the contents of webpage using GET
r = requests.get(url)
# Create Beautiful Soup object 
soup = BeautifulSoup(r.content,'html5lib')

In [41]:
# Some global variables
postcode = ''
borough = ''
neighbourhood = ''

Create Blank data frame with required columns

In [42]:
column_names = ['Postcode','Borough','Neighbourhood']
tor_post_codes = pd.DataFrame(columns=column_names)

Now, loop through the table of postal codes and append data frame

In [43]:
# Get the table object 
table = soup.find_all('table',{'class':'wikitable sortable'})

for tab in table:
# get the row of the table body
    for row in tab.tbody.find_all('tr'):
        try:
            # read cells 
            cells = row.find_all('td')

            postcode = cells[0].text
            # Some cells have hyperlink. The <a href> tag is present for them
            try:
                borough = cells[1].a.text
            except:
            # others do not have <a> tag    
                borough = cells[1].text
            
            neighbourhood = cells[2].text.strip('\n')
            # if Neighbourhood is 'Not Assigned' neighbourhood = borough
            if neighbourhood == 'Not assigned':
                neighbourhood = borough 
                
            # keep only entries that are not 'Not assigned'
            if borough != 'Not assigned':
                tor_post_codes = tor_post_codes.append( {
                    'Postcode':postcode,
                    'Borough': borough,
                    'Neighbourhood': neighbourhood
                }, ignore_index = True)
        except:
            print('Ignore Headers')

# inspect the newly created data frame
tor_post_codes.head()

Ignore Headers


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma

Use groupby function to group neighbourhoods

In [44]:
tor_post_codes = tor_post_codes.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

tor_post_codes.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [47]:
tor_post_codes.shape

(103, 3)